In [4]:
%%time
import inro.modeller as _m
import os

# Initialize the Modeller
modeller = _m.Modeller()
import_network_tool = modeller.tool("tmg.input_output.import_network_package")


parameters = {
    "IMPORT_NETWORK_FLAG":True,
    "EXPORT_NETWORK_FLAG": True,
    "mode_to_modify": "r",  
    "network_folder": "frtn_v2",  
#     "network_folder": "bus",  
    "export_network_folder": "exported_networks",
    "source_scenarios": [
        {"time_period": "AM", "scenario_id": 100},
        {"time_period": "Midday", "scenario_id": 200},
        {"time_period": "PM", "scenario_id": 300},
        {"time_period": "Evening", "scenario_id": 400},
        {"time_period": "Overnight", "scenario_id": 500},
    ],

}

def import_network(network_folder, network_file, scenario_id):
    """Imports a network package into a specified scenario."""
    network_package_path = os.path.join(network_folder, network_file)
    scenario_name = network_file[:-4]
    import_network_tool(network_package_path, scenario_id, 'OVERWRITE', True, scenario_name)
    print "Imported network for scenario {}".format(scenario_id)


if parameters["IMPORT_NETWORK_FLAG"]:
    for network_file in os.listdir(parameters["network_folder"]):
        for scenario in parameters["source_scenarios"]:
            if scenario["time_period"] in network_file:
                print("Found source network: {} for scenario {}".format(network_file, scenario["scenario_id"]))
                import_network(parameters["network_folder"], network_file, scenario["scenario_id"])
                

Found source network: AM FRTN+ V2 RL QC.nwp for scenario 100
Imported network for scenario 100
Found source network: Evening FRTN+ V2 RL QC.nwp for scenario 400
Imported network for scenario 400
Found source network: Midday FRTN+ V2 RL QC.nwp for scenario 200
Imported network for scenario 200
Found source network: PM FRTN+ V2 RL QC.nwp for scenario 300
Imported network for scenario 300
Wall time: 2min 51s


In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import inro.modeller as m
mm = m.Modeller()
gghm_api = mm.module('gghm.internal.api_interface')
from gghm_api import MatrixButler

In [24]:
parameters = {
    "time_period": "Midday",
    "options": [
        {"option": "BAU", "landuse": "HousingPP", "folder": "M:\GGHM v4\Caches\RTP\RTP_Refresh_2023\MXRTP_2051_BAU_20240215_hs" },
        {"option": "FRTN","landuse": "HousingPP", "folder": "M:\GGHM v4\Caches\RTP\RTP_Refresh_2023\MXRTP_2051_FRTN_20240223_hs"},
        {"option": "FRTN+","landuse": "HousingPP", "folder": "M:\GGHM v4\Caches\RTP\RTP_Refresh_2023\MX-RTP-20240411-HousingPP-51-FRTN_Plus-0220N"},
#         {"option": "FRTN+",
#          "landuse": "HousingPP", 
#          "folder": "M:\GGHM v4\Caches\RTP\RTP_Refresh_2023\MX-RTP-20240413-HousingPP-51-Bus_Sensitivity-0220N\MX-RTP-20240413-HousingPP-51-Bus_Sensitivity-0220N"},       
    ],
}


def get_filename(scenario, time_peirod, mode, skim_type, landuse):
    file_name = "{} {} {} {} Skim - {}.csv".format(scenario, time_peirod, mode, skim_type, landuse ) #Midday Auto Time Skim - Housing PP
    return file_name

for parameter in parameters["options"]:
    butler = MatrixButler.connect(parameter["folder"])
    auto_distance_skim = butler.load_matrix('skim.midday.auto.sov.dist', tall = True)
    auto_time_skim = butler.load_matrix('skim.midday.auto.sov.time',tall = True)
    
    
    distance = auto_distance_skim.to_frame().reset_index()
    distance.columns = ['i','j', 'distance']
    file_name = get_filename(parameter["option"], parameters["time_period"], "Auto", "Distance", parameter["landuse"] )
    distance.to_csv(file_name, index=False)
    
    time = auto_time_skim.to_frame().reset_index()
    time.columns = ['i','j', 'time']
    file_name = get_filename(parameter["option"], parameters["time_period"], "Auto", "Time", parameter["landuse"] )
    time.to_csv(file_name, index=False)
    
print "ALL DONE!!"

ALL DONE!!


In [21]:
type(distance)

pandas.core.frame.DataFrame